In [539]:
__author__ = "Tianxiang Liu"
__copyright__ = "Copyright (C) 2019 Tianxiang Liu"
__license__ = "Public Domain"
__version__ = "1.1"

In [2]:
import sklearn
import xlrd
import re


In [16]:
from urllib.parse import parse_qs, urlparse


In [3]:
fp = ("/Users/til018/Desktop/FDA/FDA_DRUG_ANALYSIS/excel_FDA.xlsx") 

In [4]:
wb = xlrd.open_workbook(fp)

In [5]:
sheet = wb.sheet_by_index(0)

In [9]:
url = []
for i in range(sheet.nrows): 
    url.append(sheet.cell_value(i, 0))

In [13]:
url=url[1:]

In [54]:
text_url=[]
gov = 'https://www.govinfo.gov/content/pkg/'

In [55]:
def split_url(p):
    s=urlparse(p).path
    return s.split('/',5)[3:]
    

In [56]:
for i in url:
    date, doc_no  = split_url(i)
    res = gov+date+'/html/'+doc_no+'.htm'
    text_url.append(res)

In [443]:
def find_drug_name(j):
    start = j.find('FDA recently approved')
    end = j.find('.',start)
    name_index = j.find('product',start)+8
    name1=j[name_index:end].strip()
    end2=j.find('is',end)
    name2=j[end+1:end2].strip()
    return (max(name1, name2,key=len))


In [444]:
def find_drug_name2(j):
            
    index1 = j.find('Determination of Regulatory Review Period for Purposes of Patent Extension')
    index2 = j.find(';',index1)
    return j[index2+1:]


In [447]:
def find_period(j):
    start = j.find('applicable regulatory review period')
    end = j.find(':',start)
    days = j[start:end]
    days = days.replace(',','')
    
    days_list = re.findall("(\d+\s\w+)", days)
    l=[]
    for i in days_list:
        if 'days' in i:
            l.append(i)

    if not l or len(l)!=3:
        l=['']*3
    #print(days_list)
    #print(l)
    return l

In [410]:
def find_patent_number(j):
    start = j.find('U.S. Patent')
    end = j.find(')',start)
    patent = j[start:end]
    patent_no = re.search("(\d+\,\d+\,\d+)", patent)
    l = patent_no.group(0) if patent_no else '' 
    return l
    
    
    
    
    
    

In [535]:
def find_date(j):
    #response = urlopen(text_url[i])
    #data = response.read().decode("utf-8")
    #blocks = re.compile(r"(\n{2,})").split(data)
    #blocks = [j.replace('\n', '') for j in blocks]
    #blocks = [j.replace('<Register>', '') for j in blocks]
    #l=['','','','']

    if ('1. The date' in j):
        start = j.find('1. The date')
        end = j.find('This determination',start)
        #date = j[start:end]
        #print(date)

        date1_end = j.find('2. The date',start)
        date1 = j[start:date1_end]

        date2_end = j.find('3. The date')
        date2 = j[date1_end:date2_end]

        date3 = j[date2_end:end]

        l = ['']*3


        date_all1 = re.findall("(\w+\s\d+\,\s\d+)", date1) if date1 else []
        date_all2 = re.findall("(\w+\s\d+\,\s\d+)", date2) if date2 else []
        date_all3 = re.findall("(\w+\s\d+\,\s\d+)", date3) if date3 else []

        l[0] = date_all1[-1] if date_all1 else '-'
        l[1] = date_all2[-1] if date_all2 else '-'
        l[2] = date_all3[-1] if date_all3 else '-'

        return l

In [310]:
write_data=[]
len(text_url)

In [327]:
import openpyxl
xfile = openpyxl.load_workbook('/Users/til018/Desktop/FDA/FDA_DRUG_ANALYSIS/excel_FDA.xlsx')

sheet1 = xfile.get_sheet_by_name('Sheet1')


In [537]:
for i in range(len(text_url)):
    response = urlopen(text_url[i])
    data = response.read().decode("utf-8")
    blocks = re.compile(r"\n{2,}").split(data)
    blocks = [j.replace('\n', '') for j in blocks]
    blocks = [j.replace('<Register>', '') for j in blocks]
    print(i)
    action =''
    date='    '
    period='   '
    drug_name1=''
    drug_name2=''
    patent_number=''

    for j in blocks:
        j.replace('\n','')
        if re.match(r'^Correction',j):
            action = j.strip()
            sheet1['H'+str(i+2)]=action
            print(j.strip())
            break
        if re.match(r'^ACTION', j):
            action = j[8:].strip()
            print(j[8:].strip())
            
        if 'Determination of Regulatory Review Period for Purposes of Patent Extension' in j and not drug_name2:
            drug_name2 = find_drug_name2(j)

        if ('FDA recently approved' in j)and not drug_name1:
            drug_name1 = find_drug_name(j)
        if ('U.S. Patent' in j):
            patent_number = find_patent_number(j)
        if ('applicable regulatory review period' in j):   
            period = find_period(j)
        if ('1. The date' in j):
            date = find_date(j)
    
    drug_name = drug_name2
    if not drug_name2:
        drug_name=drug_name1
        

    sheet1['B'+str(i+2)]=str(date[0])
    sheet1['C'+str(i+2)]=str(date[1])
    sheet1['D'+str(i+2)]=str(date[2])
    sheet1['E'+str(i+2)]='#'
    sheet1['F'+str(i+2)]=drug_name
    sheet1['G'+str(i+2)]=patent_number
    sheet1['H'+str(i+2)]=period[0] if period[0] else ''
    sheet1['I'+str(i+2)]=period[1] if period[1] else ''
    sheet1['J'+str(i+2)]=period[2] if period[2] else ''
    sheet1['K'+str(i+2)]=action
    print( date,period,drug_name,patent_number)


0
Notice.
['May 19, 1983', 'June 18, 1992', 'July 23, 1993'] ['3720 days', '3319 days', '401 days']  Betaseron 4,588,585
1
Notice.
['February 8, 1973', 'October 25, 1990', 'August 13, 1993'] ['7493 days', '6469 days', '1024 days']  Imagent GI 3,975,512
2
Notice.
['March 23, 1986', 'January 17, 1992', 'December 23, 1992'] ['2831 days', '2124 days', '707 days']  Mycobutin<SUP>TM 4,219,478
3
Notice.
          
4
Notice of proposed rulemaking.
          
5
Notice.
['October 1, 1987', 'June 28, 1991', 'December 30, 1993'] ['2284 days', '1367 days', '917 days']  Aceon<SUP>TM 4,508,729
6
Notice.
['July 14, 1982', 'October 30, 1992', 'March 4, 1994'] ['4253 days', '3762 days', '491 days']  Cattlyst 50 Type A Medicated Article 4,431,665
7
Notice.
['July 9, 1987', 'July 13, 1989', 'March 10, 1994'] ['2438 days', '736 days', '1702 days']  Aviax 4,804,680
8
Notice.
['September 6, 1984', 'December 29, 1987', 'March 25, 1994'] ['3489 days', '1210 days', '2279 days']  Semprex<SUP>TM-D Capsules 4,650,

71
Notice.
     ['5990 days', '4608 days', '1382 days']  Silvacote K 4,209,430
72
Notice.
['September 4, 1991', 'February 28, 1996', 'March 7, 1997'] ['2013 days', '1639 days', '374 days']  Skelid 4,876,248
73
Notice.
['September 27, 1991', 'June 14, 1995', 'October 29, 1996'] ['1861 days', '1357 days', '504 days']  Ray Threaded Fusion Cage<SUP>TM</SUP> 4,961,740
74
Notice.
['part 1, 98', '-', 'October 25, 1996'] ['6572 days', '5910 days', '662 days']  Rimadyl 4,264,500
75
Notice.
     ['3503 days', '1883 days', '1620 days']  Normiflo 
76
Notice.
['October 28, 1983', 'December 12, 1988', 'September 23, 1997'] ['5081 days', '1873 days', '3208 days']  Corlopam 4,197,297
77
Notice.
['part 1, 98', '-', '-'] ['2489 days', '2066 days', '423 days']  Wiktor Prime Coronary Stent System 4,886,062
78
Notice.
['August 30, 1987', 'July 26, 1996', 'February 27, 1997'] ['3471 days', '3254 days', '217 days']  Aldara<SUP>TM</SUP> (4,689,338) 
79
Notice.
     ['3471 days', '3254 days', '217 days']  Alda

142
Notice.
['August 21, 1991', 'March 31, 1998', 'August 19, 1999'] ['2922 days', '2415 days', '507 days']  Aciphex\<SUP>TM</SUP>\ 5,045,552
143
Notice.
['July 30, 1993', 'December 30, 1996', 'December 24, 1998'] ['1975 days', '1250 days', '725 days']  Provigil 4,177,290
144
Notice.
          
145
Notice.
          
146
Notice.
['April 19, 1991', 'June 26, 1996', 'June 26, 1997'] ['2262 days', '1896 days', '366 days']  Baycol 5,006,530
147
Notice.
['September 27, 1985', 'April 10, 1996', 'October 6, 1997'] ['4394 days', '3849 days', '545 days']  Infergen 
148
Notice.
          
149
Notice.
['June 12, 1988', 'November 27, 1996', 'April 23, 1999'] ['3969 days', '3091 days', '878 days']  Xenical 4,598,089
150
Notice.
['November 2, 1989', '-', '-'] ['3482 days', '684 days', '2798 days']  Uvasorb HA88 
151
Final rule.
          
152
Notice.
          
153
Notice.
['May 1, 1991', 'November 6, 1995', 'September 30, 1997'] ['2346 days', '1651 days', '695 days']  Gabitril 5,010,090
154
Notice.

219
Notice.
['March 17, 1995', 'March 8, 1999', 'November 16, 2001'] ['2438 days', '1453 days', '985 days']  GYNECARE INTERGEL 5,532,221
220
Notice.
['November 14, 1991', 'May 29, 2001', 'October 17, 2001'] ['3627 days', '3485 days', '142 days']  OP-1 IMPLANT 5,258,494
221
Notice.
['May 23, 1997', 'December 27, 2001', 'October 25, 2002'] ['1983 days', '1680 days', '303 days']  ZETIA 
222
Notice.
['February 28, 1992', 'November 1, 1999', '-'] ['3047 days', '2806 days', '241 days']  HYPERION LTK SYSTEM 
223
Notice.
['January 8, 1997', 'March 28, 2001', 'April 25, 2002'] ['1935 days', '1541 days', '394 days']  FASLODEX 4,659,516
224
Notice.
['September 8, 1995', 'November 17, 2000', 'May 24, 2002'] ['2452 days', '1898 days', '554 days']  VFEND 5,567,817
225
Notice.
['August 1, 1990', 'March 9, 1998', 'October 1, 1999'] ['3350 days', '2778 days', '572 days']  TIKOSYN 4,959,366
226
Notice.
['December 8, 1990', 'April 7, 1997', 'April 21, 2000'] ['3424 days', '2313 days', '1111 days']  EXELO

295
Notice.
          
296
Notice.
['November 23, 1996', 'May 24, 2004', 'November 23, 2004'] ['2924 days', '2740 days', '184 days']  TYSABRI 5,840,299
297
Notice.
          
298
Notice.
['March 29, 1998', 'April 29, 2002', 'March 16, 2005'] ['2546 days', '1493 days', '1053 days']  MYCAMINE 
299
Notice.
['March 7, 1996', 'March 26, 1999', 'September 24, 1999'] ['1298 days', '1115 days', '183 days']  ALAMAST 5,034,230
300
Notice.
['May 5, 1999', 'December 19, 2002', 'November 19, 2004'] ['2027 days', '1325 days', '702 days']  VESICARE 6,017,927
301
Notice.
['March 29, 1998', 'April 26, 2004', 'March 16, 2005'] ['2546 days', '2221 days', '325 days']  MYCAMINE 5,376,634
302
Notice; correction.
          FASLODEX; Correction 
303
Notice.
['November 10, 1994', 'October 29, 1999', 'August 11, 2000'] ['2103 days', '1815 days', '288 days']  CETROTIDE 5,198,533
304
Notice.
['June 13, 1992', 'February 28, 2005', 'August 30, 2005'] ['4828 days', '4644 days', '184 days']  INCRELEX 
305
Notice.
['J

...

502
Notice.
['December 19, 1998', 'December 21, 2009', 'September 21, 2010'] ['4296 days', '4021 days', '275 days']  GILENYA 5,604,229
503
Notice.
['October 30, 1998', 'March 31, 2010', 'June 17, 2010'] ['4250 days', '4171 days', '79 days']  JEVTANA 5,847,170
504
Notice.
['June 5, 1999', 'December 19, 2007', 'June 30, 2009'] ['3680 days', '3120 days', '560 days']  FERAHEME 6,599,498
505
Notice.
['March 26, 1993', 'July 30, 2010', 'September 9, 2010'] ['6378 days', '6336 days', '42 days']  EQUIDONE GEL 5,372,818
506
Notice.
['June 21, 2001', 'December 19, 2008', 'June 1, 2010'] ['3269 days', '2739 days', '530 days']  PROLIA 6,740,522
507
Notice; correction.
          KALBITOR; Correction 5,795,685
508
Notice.
['June 29, 1999', 'August 23, 2001', 'January 14, 2010'] ['3854 days', '787 days', '3067 days']  Progel Pleural Air Leak Sealant 
509
Notice.
['November 30, 2001', 'June 30, 2009', 'May 28, 2010'] ['3103 days', '2770 days', '333 days']  CYSVIEW (Previously HEXVIX) 7,247,655
510
Not

570
Notice.
['August 27, 2004', 'August 12, 2011', 'September 12, 2012'] ['2940 days', '2542 days', '398 days']  AUBAGIO--Patent No. 6,794,410 6,794,410
571
Notice.
['April 10, 2007', 'December 23, 2011', 'February 1, 2013'] ['2126 days', '1719 days', '407 days']  RAVICTI 5,968,979
572
Notice.
['July 13, 2005', 'May 29, 2012', 'November 29, 2012'] ['2698 days', '2513 days', '185 days']  COMETRIQ 7,579,473
573
Notice.
['October 22, 2004', 'April 18, 2008', 'January 16, 2013'] ['3010 days', '1275 days', '1735 days']  FLUBLOK 5,762,939
574
Notice.
['December 18, 2005', 'March 29, 2012', 'January 29, 2013'] ['2601 days', '2294 days', '307 days']  KYNAMRO 7,511,131
575
Notice.
['-', 'June 27, 2012', 'December 21, 2012'] ['', '', '']  Xience Xpedition Everolimus Eluting Coronary Stent System 7,828,766
576
Notice.
['May 19, 2004', 'November 17, 2011', 'September 4, 2012'] ['3032 days', '2739 days', '293 days']  BOSULIF 
577
Notice.
['December 11, 2009', 'December 23, 2011', 'September 28, 201

633
Notice.
['May 2, 1996', 'March 27, 2013', 'February 24, 2014'] ['6509 days', '6174 days', '335 days']  MYALEPT 6,001,968
634
Notice.
['-', '-', '-'] ['2928 days', '2009 days', '919 days']  BELVIQ 6,953,787
635
Notice.
['February 26, 1993', 'September 23, 2003', 'May 22, 2006'] ['4835 days', '3862 days', '973 days']  IONSYS 5,697,896
636
Final rule.
          
637
Notice.
['November 15, 1997', 'January 27, 2000', 'June 20, 2003'] ['2045 days', '804 days', '1241 days']  PRILOSEC OTC 5,817,338
638
Notice.
['June 14, 2009', 'August 27, 2014', 'August 27, 2015'] ['2267 days', '1901 days', '366 days']  REPATHA 8,030,457
639
Notice.
['May 23, 2007', 'May 16, 2013', 'August 15, 2014'] ['2643 days', '2186 days', '457 days']  PLEGRIDY 7,446,173
640
Notice.
          
641
Notice.
['February 1, 2004', 'September 20, 2013', 'August 19, 2014'] ['3854 days', '3520 days', '334 days']  CERDELGA 7,196,205
642
Notice.
['December 15, 2004', 'April 16, 2013', 'July 16, 2014'] ['3502 days', '3045 days',

700
Notice.
['April 12, 2013', 'March 6, 2015', 'January 8, 2016'] ['1003 days', '694 days', '309 days']  PERCEVAL SUTURELESS HEART VALVE 8,540,768
701
Notice.
['June 21, 2005', 'November 19, 2012', 'September 17, 2015'] ['3742 days', '2709 days', '1033 days']  VRAYLAR 7,737,142
702
Notice.
['April 8, 2007', 'September 5, 2014', 'September 1, 2015'] ['3070 days', '2708 days', '362 days']  VARUBI 7,049,320
703
Notice.
['October 29, 2009', 'December 22, 2014', 'December 21, 2015'] ['2246 days', '1881 days', '365 days']  UPTRAVI 7,205,302
704
Notice.
['July 7, 2009', '-', '-'] ['1688 days', '1636 days', '52 days']  GALLIPRANT 7,960,407
705
Notice.
['January 28, 1999', 'December 19, 2014', 'September 22, 2015'] ['6083 days', '5805 days', '278 days']  LONSURF 6,479,500
706
Notice.
['December 22, 2010', 'May 28, 2015', 'January 28, 2016'] ['1865 days', '1619 days', '246 days']  ZEPATIER 7,973,040
707
Notice.
['October 13, 2006', 'December 15, 2010', 'May 28, 2014'] ['2786 days', '1525 days',

765
Notice.
['May 30, 2009', 'November 16, 2016', 'July 13, 2017'] ['2968 days', '2728 days', '240 days']  TREMFYA 7,935,344
766
Notice.
['-', '-', '-'] ['', '', '']  7,041,635
767
Notice.
['-', '-', '-'] ['', '', '']  7,799,900
768
Notice.
['July 13, 2012', 'October 13, 2016', 'May 1, 2017'] ['1755 days', '1554 days', '201 days']  IMFINZI 8,779,108
769
Notice.
['January 8, 2006', 'March 30, 2016', 'April 28, 2017'] ['4130 days', '3735 days', '395 days']  TYMLOS 7,803,770
770
Notice.
['May 11, 2011', 'January 12, 2016', 'May 18, 2016'] ['1836 days', '1708 days', '128 days']  TECENTRIQ 8,217,149
771
Notice.
['February 16, 2003', 'December 20, 2016', '-'] ['5298 days', '5057 days', '241 days']  BESPONSA 8,153,768
772
Notice.
['November 9, 2003', 'September 1, 2015', 'April 29, 2016'] ['4557 days', '4315 days', '242 days']  NUPLAZID 7,601,740
773
Notice.
['August 22, 2008', 'November 16, 2015', 'February 15, 2017'] ['3101 days', '2643 days', '458 days']  SILIQ 7,767,206
774
Notice.
['June

In [538]:
xfile.save('text3.xlsx')

In [534]:
ddd1,ddd2,ddd3 = 0,0,0

for i in range(len(text_url)):
    print(i)
    response = urlopen(text_url[i])
    data = response.read().decode("utf-8")
    blocks = re.compile(r"(\n{2,})").split(data)
    blocks = [j.replace('\n', '') for j in blocks]
    blocks = [j.replace('<Register>', '') for j in blocks]
    l=['','','','']
    for j in blocks:
        if ('1. The date' in j):
            start = j.find('1. The date')
            end = j.find('This determination',start)
            #date = j[start:end]
            #print(date)

            date1_end = j.find('2. The date',start)
            date1 = j[start:date1_end]

            date2_end = j.find('3. The date')
            date2 = j[date1_end:date2_end]

            date3 = j[date2_end:end]

            l = ['']*3
            
            
            date_all1 = re.findall("(\w+\s\d+\,\s\d+)", date1) if date1 else []
            date_all2 = re.findall("(\w+\s\d+\,\s\d+)", date2) if date2 else []
            date_all3 = re.findall("(\w+\s\d+\,\s\d+)", date3) if date3 else []
            
            l[0] = date_all1[-1] if date_all1 else '-'
            l[1] = date_all2[-1] if date_all2 else '-'
            l[2] = date_all3[-1] if date_all3 else '-'
            
            print(l)
            
            
            '''
            if ('FDA records indicate') in date1:
                vf = re.search("FDA records indicate", date1)
                
                r = re.search("(\w+\s\d+\,\s\d+)", date1[vf.start():-1]) if vf else []
                l[0] = r.group(0) if r else '-'

            elif('FDA has verified') in date1:
                vf = re.search("FDA has verified", date1)
                
                r = re.search("(\w+\s\d+\,\s\d+)", date1[vf.start():-1]) if vf else []
                l[0] = r.group(0) if r else '-'
                #print(vf)
                #print(1)
                #l[0] = re.search("(\w+\s\d+\,\s\d+)", date1[vf:date1.end()])
            else:
                l[0] = '-'
                ddd1+=1
            


            if ('FDA records indicate') in date2:
                
                vf = re.search("FDA records indicate", date1)
                
                r = re.search("(\w+\s\d+\,\s\d+)", date2[vf.start():-1]) if vf else []
                l[1] = r.group(0) if r else '-'
                

            elif('FDA has verified') in date2:
                
                vf = re.search("FDA has verified", date1)
                
                r = re.search("(\w+\s\d+\,\s\d+)", date2[vf.start():-1]) if vf else []
                l[1] = r.group(0) if r else '-'
                #print(vf)
                #print(1)
                #l[0] = re.search("(\w+\s\d+\,\s\d+)", date1[vf:date1.end()])
            else:
                l[1] = re.findall("(\w+\s\d+\,\s\d+)", date2)[-1]
                


            if('FDA has verified') in date3:
                r = re.search("(\w+\s\d+\,\s\d+)", date3) 
                l[2] = r.group(0) if r else '-'
            else:
                l[2] = '-'
                ddd3+=1
            '''
            
            #print(l)
            #print(ddd1,ddd2,ddd3)



0
['May 19, 1983', 'June 18, 1992', 'July 23, 1993']
1
['February 8, 1973', 'October 25, 1990', 'August 13, 1993']
2
['March 23, 1986', 'January 17, 1992', 'December 23, 1992']
3
4
5
['October 1, 1987', 'June 28, 1991', 'December 30, 1993']
6
['July 14, 1982', 'October 30, 1992', 'March 4, 1994']
7
['July 9, 1987', 'July 13, 1989', 'March 10, 1994']
8
['September 6, 1984', 'December 29, 1987', 'March 25, 1994']
9
10
11
12
['January 14, 1988', 'September 14, 1990', 'August 5, 1994']
13
14
['October 10, 1986', 'February 26, 1991', 'November 21, 1994']
15
['June 21, 1988', 'November 10, 1994', 'May 3, 1995']
16
['July 3, 1987', 'November 15, 1993', 'May 10, 1995']
17
18
['April 20, 1988', 'November 16, 1993', 'July 21, 1995']
19
['May 24, 1990', 'December 16, 1994', 'August 2, 1995']
20
['August 1, 1987', 'December 20, 1991', 'December 20, 1995']
21
['November 23, 1984', 'May 28, 1996', 'May 31, 1996']
22
['April 8, 1985', 'July 19, 1991', 'March 19, 1996']
23
['September 20, 1989', 'July

['June 26, 1992', 'May 8, 1998', 'November 2, 1998']
186
187
188
189
['October 13, 1990', 'June 30, 1998', 'February 2, 2000']
190
['August 30, 1997', 'December 20, 1999', 'May 7, 2001']
191
['July 30, 1997', 'December 23, 1999', 'January 19, 2001']
192
['November 25, 1996', 'November 30, 1999', 'April 5, 2001']
193
194
195
['September 13, 1995', '-', '-']
196
197
['June 13, 1983', 'March 18, 1996', 'September 28, 2000']
198
['June 22, 1992', 'June 23, 1997', 'July 18, 2000']
199
['May 10, 1990', 'June 30, 1999', 'February 9, 2000']
200
['June 28, 1989', 'August 15, 1997', 'June 30, 2000']
201
['August 1, 1992', 'December 30, 1997', 'December 10, 1999']
202
['May 10, 1998', 'February 27, 2001', 'May 10, 2001']
203
['April 16, 1995', 'August 27, 1998', 'January 11, 2000']
204
['May 29, 1988', 'May 10, 1996', 'March 25, 1999']
205
['January 19, 1994', 'August 13, 1998', 'August 11, 1999']
206
['October 23, 1993', 'November 24, 1998', 'May 25, 1999']
207


KeyboardInterrupt: 

In [533]:
text_url[8]

'https://www.govinfo.gov/content/pkg/FR-1994-09-21/html/94-23276.htm'